In [1]:
import re
import tensorflow as tf
import numpy as np
import time
import os
import collections
import random
import nltk
import time
import pickle

In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return [y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0]

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global data
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [3]:
with open('books/Goblet-Of-Fire.txt', 'r',encoding = "ISO-8859-1") as fopen:
    vocabulary = clearstring(' '.join(nltk.word_tokenize(fopen.read())))
print('example 10 words:',vocabulary[:10])
print('size corpus:',len(vocabulary))
vocabulary_size = len(list(set(vocabulary)))
print('size of unique words:',vocabulary_size)
dimension = 128
skip_window = 1
num_skips = 2
batch_size = 64
location = os.getcwd()

example 10 words: ['villagers', 'little', 'hangleron', 'still', 'called', 'riddle', 'house', 'even', 'though', 'been']
size corpus: 111736
size of unique words: 10165


In [4]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('harry', 3132), ('said', 2633), ('that', 1712), ('they', 1298)]
Sample data [2651, 193, 7689, 46, 297, 984, 182, 106, 31, 17] ['villagers', 'little', 'hangleron', 'still', 'called', 'riddle', 'house', 'even', 'though', 'been']


In [5]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [6]:
graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, dimension], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, dimension],
                                                      stddev=1.0 / np.sqrt(dimension)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=batch_size / 2,
                                             num_classes=vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()

num_steps = 200000

In [7]:
with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  152.382232666
Nearest to than: lifted, dumbhedore, slaughtered, scuttled, shamrockcovered, cuthbert, performance, nott,
Nearest to face: numbers, sympathetically, chap, feast, wrath, meaningful, furious, doctors,
Nearest to looking: cares, bird, milk, scouring, marchers, plentiful, bedspread, summed,
Nearest to wand: insane, funny, milkwhite, severed, panicstricken, fetched, balanced, anyone,
Nearest to from: fiftyfoothigh, sidle, display, loosened, loomed, provide, rifled, scoffed,
Nearest to crouch: systems, raising, greatly, ofmagic, note, voluminous, steakandkidney, wilkes,
Nearest to away: scuttle, regarded, thunderclap, plotting, wheezy, member, screamed, sirs,
Nearest to george: sauce, butterbeers, awful, extracted, crashing, dreadful, filled, acclaimed,
Nearest to hogwarts: fellows, chapped, mulciber, disrepair, breezily, artfully, project, opening,
Nearest to that: block, sample, palm, balanced, suitability, arranged, also, distracted,
Ne

Average loss at step  42000 :  3.20326633847
Average loss at step  44000 :  3.1386260516
Average loss at step  46000 :  3.12206059432
Average loss at step  48000 :  3.10721587241
Average loss at step  50000 :  3.05642278832
Nearest to than: flailing, much, shamrockcovered, more, glare, hysterical, hermyohnee, defensively,
Nearest to face: numbers, message, shot, mistake, preening, notices, sickly, demonstrate,
Nearest to looking: looked, hysteria, brass, cares, middleaged, sonny, crawling, tinpot,
Nearest to wand: breeds, issuing, delusion, anger, pile, sloshing, severed, ruining,
Nearest to from: scoffed, footing, sidle, loosened, resume, cackling, crouched, into,
Nearest to crouch: systems, gamekeeper, raising, hermownninny, flutter, greatly, style, threat,
Nearest to away: member, formulating, regarded, twentyfourth, circuit, unaspected, sirs, around,
Nearest to george: dreadful, sauce, thinner, awful, windowsill, dreaming, extracted, intent,
Nearest to hogwarts: stew, swerved, sanc

Average loss at step  92000 :  2.69327366573
Average loss at step  94000 :  2.75566879243
Average loss at step  96000 :  2.69741336697
Average loss at step  98000 :  2.7178599925
Average loss at step  100000 :  2.68374176079
Nearest to than: flailing, causes, defensively, eyelid, shown, scotland, continents, much,
Nearest to face: wardrobe, soul, pensieve, demonstrate, message, slack, welcomed, lamps,
Nearest to looking: looked, hysteria, sonny, mudblood, middleaged, chivvied, smiling, redheads,
Nearest to wand: delusion, breeds, issuing, iced, concentration, irritation, severed, anger,
Nearest to from: scoffed, footing, anthem, sidle, loomed, paste, loosened, foals,
Nearest to crouch: systems, unremarkable, threat, excuse, careful, mock, supreme, losses,
Nearest to away: makeshift, member, lefthand, afford, aside, regarded, unaspected, formulating,
Nearest to george: thinner, dreadful, sauce, windowsill, awful, dreaming, wholehearted, horrors,
Nearest to hogwarts: sanctimoniously, ste

Average loss at step  142000 :  2.52841079992
Average loss at step  144000 :  2.50773842812
Average loss at step  146000 :  2.54137210977
Average loss at step  148000 :  2.4661333822
Average loss at step  150000 :  2.55190739185
Nearest to than: eyelid, causes, defensively, scotland, shown, piled, glare, more,
Nearest to face: wardrobe, welcomed, litter, slack, preening, pince, message, pensieve,
Nearest to looking: looked, sonny, appearing, apologies, crawling, connecting, hysteria, chivvied,
Nearest to wand: concentration, delusion, breeds, issuing, splattered, impostor, cleaned, severed,
Nearest to from: scoffed, footing, stinking, anthem, grumpily, wispylooking, midriff, slide,
Nearest to crouch: systems, unremarkable, hermione, hermownninny, careful, excuse, congratulations, threat,
Nearest to away: regarded, makeshift, afford, lefthand, scandalized, halfbow, specifically, rate,
Nearest to george: thinner, dreadful, sauce, windowsill, wholehearted, dreaming, awful, autograph,
Near

Average loss at step  192000 :  2.47233164281
Average loss at step  194000 :  2.40411202824
Average loss at step  196000 :  2.42580278414
Average loss at step  198000 :  2.41513011521


In [8]:
with open('goblet-list.p', 'wb') as fopen:
    pickle.dump(list(reverse_dictionary.values()), fopen)

In [9]:
with open('goblet-vector.p', 'wb') as fopen:
    pickle.dump(final_embeddings, fopen)